<a href="https://colab.research.google.com/github/samibahig/IFT6135/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
import torchvision.transforms


from_numpy = torch.from_numpy
batch_size = 32
num_epochs = 10
store_every = 700
lr0 = 0.03
#model_type = 'MLP'
model_type = 'CNN'
mnist_transforms = torchvision.transforms.Compose(
        [torchvision.transforms.ToTensor()])
mnist_train = torchvision.datasets.MNIST(
        root='./data', train=True, 
        transform=mnist_transforms, download=True)
mnist_test = torchvision.datasets.MNIST(
        root='./data', train=False, 
        transform=mnist_transforms, download=True)

train_loader = torch.utils.data.DataLoader(
        mnist_train, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(
        mnist_test, batch_size=batch_size, shuffle=True, num_workers=2)

class ResLinear(nn.Module):

    def __init__(self, in_features, out_features, activation=nn.ReLU()):
        super(ResLinear, self).__init__()
        
        self.in_features = in_features
        self.out_features = out_features
        self.activation = activation
        
        self.linear = nn.Linear(in_features, out_features)
        if in_features != out_features:
            self.project_linear = nn.Linear(in_features, out_features)
        
    def forward(self, x):
        inner = self.activation(self.linear(x))
        if self.in_features != self.out_features:
            skip = self.project_linear(x)
        else:
            skip = x
        return inner + skip


class Flatten(nn.Module):
    def forward(self, x):
        x = x.view(x.size(0), -1)
        return x

if model_type == 'CNN':
  model = nn.Sequential(nn.Conv2d(1,100,3),
        nn.ReLU(),
        nn.MaxPool2d(2),
        nn.Conv2d(100,122,5),
        nn.ReLU(),
        nn.MaxPool2d(2),
        Flatten(),
        ResLinear(1952, 100),
        nn.ReLU(),
        ResLinear(100, 10)
    )

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr0)

#Learning rate
def adjust_lr(optimizer, epoch, total_epochs):
    lr = lr0 * (0.1 ** (epoch / float(total_epochs)))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

#Evalution Functions
def accuracy(proba, y):
    correct = torch.eq(proba.max(1)[1], y).sum().type(torch.FloatTensor)
    return correct / y.size(0)
    
    
def evaluate(dataset_loader, criterion):
    LOSSES = 0
    COUNTER = 0
    for batch in dataset_loader:
        optimizer.zero_grad()
        x,y = batch
        if model_type == 'CNN':
            x = x.view(-1,1,28,28)
            y = y.view(-1)
          
        loss = criterion(model(x), y)
        n = y.size(0)
        LOSSES += loss.sum().data.cpu().numpy() * n
        COUNTER += n
    return LOSSES / float(COUNTER)

#Training loop
LOSSES = 0
COUNTER = 0
ITERATIONS = 0
train_losses = [] # save loss
train_accuracy =[] #save accuracy
tess_accuracy =[] #save accuracy
test_losses =[] #save accuracy

learning_curve_nll_train = list()
learning_curve_nll_test = list()
learning_curve_acc_train = list()
learning_curve_acc_test = list()
for e in range(num_epochs):
    print('Epoch', e+1)
    for batch in train_loader:
        optimizer.zero_grad()
        x,y = batch
        if model_type == 'CNN':
            x = x.view(-1,1,28,28)
            y = y.view(-1)
            
        loss = criterion(model(x), y)
        loss.backward()
        optimizer.step()
        
        n = y.size(0)
        LOSSES += loss.sum().data.cpu().numpy() * n
        COUNTER += n
        ITERATIONS += 1
        if ITERATIONS%(store_every/5) == 0:
            avg_loss = LOSSES / float(COUNTER)
            LOSSES = 0
            COUNTER = 0
            print(" Iteration {}: TRAIN {}".format(
                ITERATIONS, avg_loss))
    
        if ITERATIONS%(store_every) == 0 or ITERATIONS%(938) == 0 :     
            train_loss = evaluate(train_loader, criterion)
            learning_curve_nll_train.append(train_loss)
            test_loss = evaluate(test_loader, criterion)
            learning_curve_nll_test.append(test_loss)

            train_acc = evaluate(train_loader, accuracy)
            learning_curve_acc_train.append(train_acc)
            test_acc = evaluate(test_loader, accuracy)
            learning_curve_acc_test.append(test_acc)
                    
            print(" [NLL] TRAIN {} / TEST {}".format(
                train_loss, test_loss))
            print(" [ACC] TRAIN {} / TEST {}".format(
                train_acc, test_acc))
    
    train_accuracy.append(train_acc)
    train_losses.append(train_loss)
    tess_accuracy.append(test_acc)
    test_losses.append(test_loss)
    print('Train accuracy', train_accuracy)
    print('Train Loss', train_loss)
    print('Train test accurary', test_loss)
    print
    print('Train accuracy', test_acc)
    print(ITERATIONS)

    
    adjust_lr(optimizer, e+1, num_epochs)